# Computational Methods in Simulation (CMIS) Week 5
This notebook is intended to support your learning process. It contains exercises that help you to understand the material. 

This week is based on Slides 19 - follow the detailed steps below to learn about the finite element method:

* Make an implementation of the 2D elastic assembly process
* Discuss from a computer science viewpoint which steps are the same and which steps that are different in the 2D elastic case compared to the 2D and 1D Poisson problems done in week 4.
* Speculate how to write a general FEM computer program that could simulate any partial differential equation with point wise boundary conditions.
* For the 2D elasticity problem, 
$\int_{\Omega^e}\left(\delta \varepsilon\right)^T\sigma d\Omega = \int_{\Gamma^e}\left(\delta u \right)^T f dS $
where $f$ is a given constant load (force per meter) use the FEM method to derive the $K_e$ and $f_e$ terms such that $K_e\,u_e =f_e$
* Based on your FEM derivation implement a rectangular solid 2D bar of dimension 6-by-2 meters and made of steel like material. Attach the left side of the bar to a wall (use boundary condition $u = 0$) and apply a downward nodal load on the right edge of the bar.    
* Examine the eigenvalue spectrum of the global stiffness matrix before applying boundary conditions and after (Hint: write down how many zero eigenvalues you expect before and after applying boundary conditions and explain how the minimum eigenvalue is related to the boundary conditions after having applied these).
* Examine the fill pattern of the stiffness matrix (Hint: reflect about the shape of the fill-in and how the non-zero values in $K$ is related to the mesh).
* Investigate what happens to the deformations when you vary the resolution and load on the bar. Try to find the best suitable mesh resolution. (Hint: it is critical that you have the right formula for $f_e$ and do not use a “constant” load per node value, can you explain why?)
* From everyday life steel material appears to be noticeable incompressible. Examine whether your implementation has volume (area in 2D) conservation (Hint: Explain the causes for your observations, consider carefully what could have “gone” wrong).

## Detailed Expected Learning Objectives of this Week

* Work with concepts such as tensors when deriving equations (such as stress and strain fields).
* Derive a FEM method for complex problem such as linear elasticity.
* Apply FEM to a more complex problem such as linear elasticity.
* Apply experimental validation to a FEM implementation of for instance linear elasticity.

In [1]:
%matplotlib widget

In [2]:
import igl
import meshplot as mp
import numpy as np
import matplotlib.pyplot as plt

## Create the Computational Mesh
We will re-use the beam mesh generator from week 3

In [3]:
def make_beam_mesh(width, height, shape):
    x0 = -width/2.0
    y0 = -height/2.0
    I  = shape[0]
    J  = shape[1]
    dx = width/float(I)
    dy = height/float(J)
    V = np.zeros(((I+1)*(J+1),2),dtype=np.float64)
    for j in range(J+1):
        for i in range(I+1):
            k = i + j*(I+1)
            V[k,0] = x0 + i*dx
            V[k,1] = y0 + j*dy
    T = np.zeros((2*I*J,3),dtype=np.int32)
    for j in range(J):
        for i in range(I):
            k00 = (i  ) + (j  )*(I+1)
            k01 = (i+1) + (j  )*(I+1)
            k10 = (i  ) + (j+1)*(I+1)
            k11 = (i+1) + (j+1)*(I+1)
            e = 2*(i + j*I)
            if (i+j)%2:
                T[e,  :] = (k00,k01,k11)
                T[e+1,:] = (k00,k11,k10)
            else:
                T[e,  :] = (k10,k00,k01)
                T[e+1,:] = (k10,k01,k11)                    
    return V, T

V, T = make_beam_mesh(6.0,2.0,(12,6))

We will also reuse the triangle area computation function we used in week 4

In [4]:
def compute_triangle_areas(V,T):
    E = len(T) # Total number of triangles in the mesh
    A = np.zeros((E,),dtype=np.float64)
    for e in range(E):
        # Get triangle indices
        i = T[e,0]
        j = T[e,1]
        k = T[e,2]
        # Get triangle coordinates
        xi = V[i,0]
        xj = V[j,0]
        xk = V[k,0]
        yi = V[i,1]
        yj = V[j,1]
        yk = V[k,1]    
        
        dx1 = xk - xj
        dy1 = yk - yj
        dx2 = xi - xj
        dy2 = yi - yj

        A[e] =  (dx1*dy2 - dx2*dy1 ) / 2.0
    return A

A = compute_triangle_areas(V,T)

# Linear elasticity FEM Theory Summary

The continous displacement field is given by interpolation of the discrete nodal displacement field

 $u(x) = N  U_e$
 
This is a 2-by-1 where 

$N = [ w_1 I, w_2 I, w_3 I ]$

This is 2-by-6 and

$U_e = [Ux(T(k,:));  Uy(T(k,:))]$

This is 6-by-one  and linear interpolation is used (bary-centric coordinates) 

   $w_1  =  area(x,2,3)  / A(k)$
   
   $w_2  =  area(3,1,x)  / A(k)$
   
   $w_3  =  area(x,1,2)  / A(k)$

The Cauchy strain tensor is given by

$\varepsilon(x) = S \, u(x) = S \, N \, U_e  = B \, U_e$

This is 3-by-1 where 

$B = S * N$

This is 3-by-6 and the differential operator is defined as

$S = \begin{bmatrix}dx & 0\\ 0 & dy\\ dy & dx\end{bmatrix}$
 
This is 3-by-2. The Cauchy Stress tensor can by found by the equation of state (EOS)
which defines a relation between stress and strain. For a neo-hookean
material model this yields the relation

$\sigma =  D \varepsilon$

This is 3-by-1 where

$D =  \frac{E}{1-\nu^2} \begin{bmatrix} 1 & \nu & 0\\
                     \nu &1 &0\\
                      0 & 0 & \frac{1-\nu}{2}\end{bmatrix}$

This is 3-by-3. Now by using the virtual work one ends up with a linear system for the unknown nodal displacements

$K_e U_e = F_e$

where

$K_e = B^T  D  B  A_e$

This is 6-by-6. To assemble the element-wise equations into one large simultaneous system one applies Newton's third law at the nodes.

In [5]:
N = len(V) # Total number of nodes in the mesh
E = len(T) # Total number of triangles in the mesh

Ke = np.zeros((6,E*6), dtype=np.float64)
K  = np.zeros( (N*2,N*2), dtype=np.float64)

# Compute the D (elasticity) matrix
#
# http://en.wikipedia.org/wiki/Young%27s_modulus
# http://en.wikipedia.org/wiki/Poisson%27s_ratio
#
# This is aluminium like parameters
Ey  = 69e9  # Young modulus
nu = 0.3    # Poisson ration

# TODO - Compute the D (elasticity) matrix
D  = 

# Now compute element stiffness matrices
for e in range(E):
    # Get triangle indices
    i = T[e,0]  
    j = T[e,1]  
    k = T[e,2]  
    # Get triangle coordinates
    xi = V[i,0]
    xj = V[j,0]
    xk = V[k,0]
    yi = V[i,1]
    yj = V[j,1]
    yk = V[k,1]
    # TODO - Compute spatial gradients of the barycentric coordinates
    
    # TODO - Compute the B matrix
           
    # TODO - Compute element stiffness matrix and store it in Ke array
    Ke[:, 6*e:6*e+6] = 

In [6]:
# Now do assembly process of global stiffness matrix
for e in range(E):
    # Get global triangle vertex indices
    i = T[e,0]  
    j = T[e,1]  
    k = T[e,2]    
    # Local order of vertex coordinates is i_x, i_y, j_x j_y, k_x, and  k_y. 
    # This is how local vertex indices (0,1,2,..,5) are mapped to global vertex
    # indices
    gidx = [ i, N + i, j,  N + j,  k, N + k]
    # Now we can add the element stiffness matrix to the global matrix using
    # our local-global vertex index mapping.
    
    # TODO - assemble Ke into global K matrix

We will visualize the assembly process

In [7]:
fig = plt.figure()
ax = plt.subplot(111)
plt.spy(K);
ax.set_title('Fill pattern of K matrix');
ax.set_ylabel('Row index');
ax.set_xlabel('Column index');
plt.show()

fig = plt.figure()
d, _ = np.linalg.eig(K)
plt.plot(np.sort( d ) );
ax.set_title('Eigenvalues of K matrix');
ax.set_xlabel('Eigenvalue Index')
ax.set_ylabel('Value');
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Create an  external nodal force vector with a prescribed load at the right hand side of the bar.

In [8]:
load = -10e7
f = np.zeros((2*N,),dtype=np.float64)
indices = np.array(np.where(V[:,0]>2.9),dtype=np.int).flatten() + N
f[ indices ] =  load

Apply Direchlet boundary conditions to displacement field for the left vertices.

In [9]:
xindices = np.array(np.where( V[:,0] < -2.9), dtype=np.int).flatten()
yindices = xindices + N

indices = np.hstack((xindices, yindices))
values  = np.zeros(indices.shape,dtype=np.float64)

F = np.setdiff1d(np.arange(2*N), indices)

for i in range(len(indices)):
    index = indices[i]
    value = values[i]  
  
    # TODO - add boundary conditions here

After having applied the boundary conditions we can now solve for the displacement field

In [10]:
u = np.zeros(f.shape, dtype=np.float64)
u[indices] = values
KFF = K[F,:][:,F]
fF  = f[F]
u[F] = np.linalg.solve(KFF, fF)

From displacements we can compute the spatial deformed coordinates and visualize the deformed mesh.

In [11]:
x = V[:,0] + u[0:N]
y = V[:,1] + u[N:2*N]

fig = plt.figure()
plt.triplot(V[:,0],V[:,1],T)
plt.triplot(x,y,T)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

That is all folks